In [4]:
import os
import pandas as pd
import requests
import shutil
import lxml.html
from lxml.cssselect import CSSSelector

In [85]:
pokemon = 'Aerodactyl'

In [86]:
response = requests.get(f'https://www.pokemon.com/es/pokedex/{pokemon.lower()}')
img_selector = CSSSelector('.profile-images img')
img_results = img_selector(lxml.html.fromstring(response.text))
image_url = [result.attrib['src'] for result in img_results][0]
image_url

'https://assets.pokemon.com/assets/cms2/img/pokedex/full/142.png'

In [70]:
number_selector = CSSSelector('.pokedex-pokemon-pagination-title .pokemon-number')
number_results = number_selector(lxml.html.fromstring(response.text))
number = [result.text_content().replace('\n', ' ') for result in number_results]
number = int(number[0].split('º')[-1])
number

# .pokedex-pokemon-pagination-title .pokemon-number , .gauge li , .middle a , .background-color-electric a

145

In [39]:
desc_selector = CSSSelector('p.version-y')
desc_results = desc_selector(lxml.html.fromstring(response.text))
desc = [' '.join(result.text.replace('\n', ' ').split()) for result in desc_results][0]
desc

'Zapdos es un Pokémon pájaro legendario que tiene la habilidad de controlar la electricidad. Suele vivir en nubarrones. Este Pokémon gana mucha fuerza si le alcanzan los rayos.'

In [97]:
info_selector = CSSSelector('.column-7')
info_results = info_selector(lxml.html.fromstring(response.text))
info = ' '.join([' '.join(result.text_content().replace('\n', ' ').split()) for result in info_results]).split()
if 'Altura' in info[1:]:
    info = info[:info[1:].index('Altura')]
print(info)
info = {
    'Altura': ' '.join([info[1], info[2]]),
    'Peso': ' '.join([info[4], info[5]]),
    'Habilidad': ' '.join(info[info.index('Habilidad')+1:])
}
info

['Altura', '1,8', 'm', 'Peso', '59,0', 'kg', 'Sexo', 'Categoría', 'Fósil', 'Habilidad', 'Cabeza', 'Roca']


{'Altura': '1,8 m', 'Habilidad': 'Cabeza Roca', 'Peso': '59,0 kg'}

In [44]:
types_selector = CSSSelector('.dtm-type ul')
types_results = types_selector(lxml.html.fromstring(response.text))
types = [result.text_content().replace('\n', ' ').split() for result in types_results][0]
types

['Eléctrico', 'Volador']

In [64]:
stats_list = ['HP', 'Ataque', 'Ataque Sp.', 'Defensa', 'Defensa Sp.', 'Velocidad']
stats_response = requests.get(f'http://pokeapi.co/api/v2/pokemon/{25}').json()
stats = {stats_list[int(stat['stat']['url'].split('/')[-2]) - 1]: stat['base_stat'] \
                     for stat in stats_response['stats']}
stats

{'Ataque': 55,
 'Ataque Sp.': 40,
 'Defensa': 50,
 'Defensa Sp.': 50,
 'HP': 35,
 'Velocidad': 90}

In [77]:
classes = sorted(['Bulbasaur', 'Venusaur', 'Charmander', 'Charizard', 'Squirtle',
           'Blastoise', 'Pikachu', 'Eevee', 'Moltres', 'Zapdos', 
           'Articuno', 'Mewtwo', 'Lapras', 'Dragonite',
           'Alakazam', 'Gengar', 'Snorlax', 'Gyarados', 'Arcanine',
           'Jolteon', 'Vaporeon', 'Flareon', 'Scyther',
           'Pidgeot', 'Starmie', 'Magneton', 'Aerodactyl'])

In [71]:
pokemon = 'Zapdos'

In [139]:
def edit_info(pokemon):
    response = requests.get(f'https://www.pokemon.com/es/pokedex/{pokemon.lower()}')
    abilities_selector = CSSSelector('.moreInfo .attribute-value')
    abilities_result = abilities_selector(lxml.html.fromstring(response.text))
    abilities = [' '.join(result.text_content().replace('\n', ' ').split()) for result in abilities_result]
    return abilities

In [138]:
edit_info('Blastoise')

['Torrente', 'Megadisparador']


In [131]:
edit_info('Aerodactyl')

['Cabeza Roca', 'Presión', 'Garra Dura']


In [74]:
def get_info(pokemon):
    response = requests.get(f'https://www.pokemon.com/es/pokedex/{pokemon.lower()}')
    info_selector = CSSSelector('.column-7')
    info_results = info_selector(lxml.html.fromstring(response.text))
    info = ' '.join([' '.join(result.text_content().replace('\n', ' ').split()) for result in info_results]).split()
    if 'Altura' in info[1:]:
        info = info[:info[1:].index('Altura')]
    info = {
        'Altura': ' '.join([info[1], info[2]]),
        'Peso': ' '.join([info[4], info[5]]),
    }
    abilities_selector = CSSSelector('.moreInfo .attribute-value')
    abilities_result = abilities_selector(lxml.html.fromstring(response.text))
    abilities = [' '.join(result.text_content().replace('\n', ' ').split()) for result in abilities_result]
    info['Habilidad'] = abilities
    number_selector = CSSSelector('.pokedex-pokemon-pagination-title .pokemon-number')
    number_results = number_selector(lxml.html.fromstring(response.text))
    number = [result.text_content().replace('\n', ' ') for result in number_results]
    number = int(number[0].split('º')[-1])
    info['Number'] = number
    desc_selector = CSSSelector('p.version-y')
    desc_results = desc_selector(lxml.html.fromstring(response.text))
    desc = [' '.join(result.text.replace('\n', ' ').split()) for result in desc_results][0]
    info['Description'] = desc
    img_selector = CSSSelector('.profile-images img')
    img_results = img_selector(lxml.html.fromstring(response.text))
    image_url = [result.attrib['src'] for result in img_results][0]
    info['ImageUrl'] = image_url
    types_selector = CSSSelector('.dtm-type ul')
    types_results = types_selector(lxml.html.fromstring(response.text))
    types = [result.text_content().replace('\n', ' ').split() for result in types_results][0]
    info['Types'] = types
    stats_list = ['HP', 'Ataque', 'Ataque Sp.', 'Defensa', 'Defensa Sp.', 'Velocidad']
    stats_response = requests.get(f'http://pokeapi.co/api/v2/pokemon/{number}').json()
    stats = {stats_list[int(stat['stat']['url'].split('/')[-2]) - 1]: stat['base_stat'] \
                         for stat in stats_response['stats']}
    info['Stats'] = stats
    info['Pokemon'] = pokemon
    return info    

In [84]:
get_info('Aerodactyl')

{'Altura': '1,8 m',
 'Description': 'Los orígenes de Aerodactyl datan de la era de los dinosaurios. Se regeneró a partir de material genético contenido en ámbar. Se supone que fue el amo de los cielos en épocas pasadas.',
 'Habilidad': 'Cabeza Roca Presión Altura 2,1 m Peso 79,0 kg Sexo Desconocido Categoría Fósil Habilidad Garra Dura',
 'ImageUrl': 'https://assets.pokemon.com/assets/cms2/img/pokedex/full/142.png',
 'Number': 142,
 'Peso': '59,0 kg',
 'Stats': {'Ataque': 105,
  'Ataque Sp.': 65,
  'Defensa': 60,
  'Defensa Sp.': 75,
  'HP': 80,
  'Velocidad': 130},
 'Types': ['Roca', 'Volador']}

In [78]:
data = {pokemon: get_info(pokemon) for pokemon in classes}

In [80]:
import pandas as pd

In [93]:
df['Stats']['Aerodactyl']

{'Ataque': 105,
 'Ataque Sp.': 65,
 'Defensa': 60,
 'Defensa Sp.': 75,
 'HP': 80,
 'Velocidad': 130}

In [107]:
s = pd.Series(df.index)

In [140]:
s[:4].apply(edit_info)

0    [Cabeza Roca, Presión, Garra Dura]
1     [Foco Interno, Sincronía, Rastro]
2         [Intimidación, Absorbe Fuego]
3                             [Presión]
dtype: object

In [117]:
df['Pokemon'] = df.index
# df = df.drop('index', axis=1)
df = df.reset_index(drop=True)

In [141]:
df['Habilidad'] = df['Pokemon'].apply(edit_info)

In [142]:
# df = pd.DataFrame(data).transpose()
df

,Altura,Description,Habilidad,ImageUrl,Number,Peso,Stats,Types,Pokemon
0,"1,8 m",Los orígenes de Aerodactyl datan de la era de ...,"[Cabeza Roca, Presión, Garra Dura]",https://assets.pokemon.com/assets/cms2/img/pok...,142,"59,0 kg","{'Velocidad': 130, 'Defensa Sp.': 75, 'Defensa...","[Roca, Volador]",Aerodactyl
1,"1,5 m",El cerebro de Alakazam nunca deja de crecer y ...,"[Foco Interno, Sincronía, Rastro]",https://assets.pokemon.com/assets/cms2/img/pok...,65,"48,0 kg","{'Velocidad': 120, 'Defensa Sp.': 95, 'Defensa...",[Psíquico],Alakazam
2,"1,9 m",Arcanine es conocido por lo veloz que es. Dice...,"[Intimidación, Absorbe Fuego]",https://assets.pokemon.com/assets/cms2/img/pok...,59,"155,0 kg","{'Velocidad': 95, 'Defensa Sp.': 80, 'Defensa'...",[Fuego],Arcanine
3,"1,7 m",Articuno es un Pokémon pájaro legendario que p...,[Presión],https://assets.pokemon.com/assets/cms2/img/pok...,144,"55,4 kg","{'Velocidad': 85, 'Defensa Sp.': 125, 'Defensa...","[Hielo, Volador]",Articuno
4,"1,6 m",Blastoise lanza chorros de agua con gran preci...,"[Torrente, Megadisparador]",https://assets.pokemon.com/assets/cms2/img/pok...,9,"85,5 kg","{'Velocidad': 78, 'Defensa Sp.': 105, 'Defensa...",[Agua],Blastoise
5,"0,7 m",A Bulbasaur es fácil verle echándose una siest...,[Espesura],https://assets.pokemon.com/assets/cms2/img/pok...,1,"6,9 kg","{'Velocidad': 45, 'Defensa Sp.': 65, 'Defensa'...","[Planta, Veneno]",Bulbasaur
6,"1,7 m",Charizard se dedica a volar por los cielos en ...,"[Mar Llamas, Garra Dura, Sequía]",https://assets.pokemon.com/assets/cms2/img/pok...,6,"90,5 kg","{'Velocidad': 100, 'Defensa Sp.': 85, 'Defensa...","[Fuego, Volador]",Charizard
7,"0,6 m",La llama que tiene en la punta de la cola arde...,[Mar Llamas],https://assets.pokemon.com/assets/cms2/img/pok...,4,"8,5 kg","{'Velocidad': 65, 'Defensa Sp.': 50, 'Defensa'...",[Fuego],Charmander
8,"2,2 m",Dragonite es capaz de dar la vuelta al mundo e...,[Foco Interno],https://assets.pokemon.com/assets/cms2/img/pok...,149,"210,0 kg","{'Velocidad': 80, 'Defensa Sp.': 100, 'Defensa...","[Dragón, Volador]",Dragonite
9,"0,3 m",La configuración genética de Eevee le permite ...,"[Fuga, Adaptable]",https://assets.pokemon.com/assets/cms2/img/pok...,133,"6,5 kg","{'Velocidad': 55, 'Defensa Sp.': 65, 'Defensa'...",[Normal],Eevee


In [155]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 9 columns):
Altura         27 non-null object
Description    27 non-null object
Habilidad      27 non-null object
ImageUrl       27 non-null object
Number         27 non-null object
Peso           27 non-null object
Stats          27 non-null object
Types          27 non-null object
Pokemon        27 non-null object
dtypes: object(9)
memory usage: 2.0+ KB


In [143]:
df.to_csv('Pokedex_data.csv', index=False)

In [147]:
df2 = df.copy()
df2.index = df2['Pokemon']
df2.transpose()

Pokemon,Aerodactyl,Alakazam,Arcanine,Articuno,Blastoise,Bulbasaur,Charizard,Charmander,Dragonite,Eevee,...,Moltres,Pidgeot,Pikachu,Scyther,Snorlax,Squirtle,Starmie,Vaporeon,Venusaur,Zapdos
Altura,"1,8 m","1,5 m","1,9 m","1,7 m","1,6 m","0,7 m","1,7 m","0,6 m","2,2 m","0,3 m",...,"2,0 m","1,5 m","0,4 m","1,5 m","2,1 m","0,5 m","1,1 m","1,0 m","2,0 m","1,6 m"
Description,Los orígenes de Aerodactyl datan de la era de ...,El cerebro de Alakazam nunca deja de crecer y ...,Arcanine es conocido por lo veloz que es. Dice...,Articuno es un Pokémon pájaro legendario que p...,Blastoise lanza chorros de agua con gran preci...,A Bulbasaur es fácil verle echándose una siest...,Charizard se dedica a volar por los cielos en ...,La llama que tiene en la punta de la cola arde...,Dragonite es capaz de dar la vuelta al mundo e...,La configuración genética de Eevee le permite ...,...,Moltres es un Pokémon pájaro legendario que ti...,El plumaje de este Pokémon es bonito e hipnóti...,Cada vez que un Pikachu se encuentra con algo ...,Es espectacular ver lo rápido que es Scyther. ...,Un día cualquiera en la vida de Snorlax consis...,El caparazón de Squirtle no le sirve de protec...,"La parte central de Starmie, el núcleo brillan...",Vaporeon sufrió una mutación repentina y desar...,"Venusaur tiene una flor enorme en el lomo que,...",Zapdos es un Pokémon pájaro legendario que tie...
Habilidad,"[Cabeza Roca, Presión, Garra Dura]","[Foco Interno, Sincronía, Rastro]","[Intimidación, Absorbe Fuego]",[Presión],"[Torrente, Megadisparador]",[Espesura],"[Mar Llamas, Garra Dura, Sequía]",[Mar Llamas],[Foco Interno],"[Fuga, Adaptable]",...,[Presión],"[Vista Lince, Tumbos, Indefenso]",[Elec. Estática],"[Enjambre, Experto]","[Sebo, Inmunidad]",[Torrente],"[Cura Natural, Iluminación]",[Absorbe Agua],"[Espesura, Sebo]",[Presión]
ImageUrl,https://assets.pokemon.com/assets/cms2/img/pok...,https://assets.pokemon.com/assets/cms2/img/pok...,https://assets.pokemon.com/assets/cms2/img/pok...,https://assets.pokemon.com/assets/cms2/img/pok...,https://assets.pokemon.com/assets/cms2/img/pok...,https://assets.pokemon.com/assets/cms2/img/pok...,https://assets.pokemon.com/assets/cms2/img/pok...,https://assets.pokemon.com/assets/cms2/img/pok...,https://assets.pokemon.com/assets/cms2/img/pok...,https://assets.pokemon.com/assets/cms2/img/pok...,...,https://assets.pokemon.com/assets/cms2/img/pok...,https://assets.pokemon.com/assets/cms2/img/pok...,https://assets.pokemon.com/assets/cms2/img/pok...,https://assets.pokemon.com/assets/cms2/img/pok...,https://assets.pokemon.com/assets/cms2/img/pok...,https://assets.pokemon.com/assets/cms2/img/pok...,https://assets.pokemon.com/assets/cms2/img/pok...,https://assets.pokemon.com/assets/cms2/img/pok...,https://assets.pokemon.com/assets/cms2/img/pok...,https://assets.pokemon.com/assets/cms2/img/pok...
Number,142,65,59,144,9,1,6,4,149,133,...,146,18,25,123,143,7,121,134,3,145
Peso,"59,0 kg","48,0 kg","155,0 kg","55,4 kg","85,5 kg","6,9 kg","90,5 kg","8,5 kg","210,0 kg","6,5 kg",...,"60,0 kg","39,5 kg","6,0 kg","56,0 kg","460,0 kg","9,0 kg","80,0 kg","29,0 kg","100,0 kg","52,6 kg"
Stats,"{'Velocidad': 130, 'Defensa Sp.': 75, 'Defensa...","{'Velocidad': 120, 'Defensa Sp.': 95, 'Defensa...","{'Velocidad': 95, 'Defensa Sp.': 80, 'Defensa'...","{'Velocidad': 85, 'Defensa Sp.': 125, 'Defensa...","{'Velocidad': 78, 'Defensa Sp.': 105, 'Defensa...","{'Velocidad': 45, 'Defensa Sp.': 65, 'Defensa'...","{'Velocidad': 100, 'Defensa Sp.': 85, 'Defensa...","{'Velocidad': 65, 'Defensa Sp.': 50, 'Defensa'...","{'Velocidad': 80, 'Defensa Sp.': 100, 'Defensa...","{'Velocidad': 55, 'Defensa Sp.': 65, 'Defensa'...",...,"{'Velocidad': 90, 'Defensa Sp.': 85, 'Defensa'...","{'Velocidad': 101, 'Defensa Sp.': 70, 'Defensa...","{'Velocidad': 90, 'Defensa Sp.': 50, 'Defensa'...","{'Velocidad': 105, 'Defensa Sp.': 80, 'Defensa...","{'Velocidad': 30, 'Defensa Sp.': 110, 'Defensa...","{'Velocidad': 43, 'Defensa Sp.': 64, 'Defensa'...","{'Velocidad': 115, 'Defensa Sp.': 85, 'D

In [162]:
a = df2.loc['Aerodactyl'].to_dict()
a

{'Altura': '1,8 m',
 'Description': 'Los orígenes de Aerodactyl datan de la era de los dinosaurios. Se regeneró a partir de material genético contenido en ámbar. Se supone que fue el amo de los cielos en épocas pasadas.',
 'Habilidad': ['Cabeza Roca', 'Presión', 'Garra Dura'],
 'ImageUrl': 'https://assets.pokemon.com/assets/cms2/img/pokedex/full/142.png',
 'Number': 142,
 'Peso': '59,0 kg',
 'Pokemon': 'Aerodactyl',
 'Stats': {'Ataque': 105,
  'Ataque Sp.': 65,
  'Defensa': 60,
  'Defensa Sp.': 75,
  'HP': 80,
  'Velocidad': 130},
 'Types': ['Roca', 'Volador']}

In [163]:
dict((k, a[k]) for k in ['Altura', 'Description', 'Habilidad', 'ImageUrl', 'Peso', 'Stats', 'Types'])

{'Altura': '1,8 m',
 'Description': 'Los orígenes de Aerodactyl datan de la era de los dinosaurios. Se regeneró a partir de material genético contenido en ámbar. Se supone que fue el amo de los cielos en épocas pasadas.',
 'Habilidad': ['Cabeza Roca', 'Presión', 'Garra Dura'],
 'ImageUrl': 'https://assets.pokemon.com/assets/cms2/img/pokedex/full/142.png',
 'Peso': '59,0 kg',
 'Stats': {'Ataque': 105,
  'Ataque Sp.': 65,
  'Defensa': 60,
  'Defensa Sp.': 75,
  'HP': 80,
  'Velocidad': 130},
 'Types': ['Roca', 'Volador']}

In [157]:
a = df2.loc['Aerodactyl'].to_json()
a

'{"Altura":"1,8 m","Description":"Los or\\u00edgenes de Aerodactyl datan de la era de los dinosaurios. Se regener\\u00f3 a partir de material gen\\u00e9tico contenido en \\u00e1mbar. Se supone que fue el amo de los cielos en \\u00e9pocas pasadas.","Habilidad":["Cabeza Roca","Presi\\u00f3n","Garra Dura"],"ImageUrl":"https:\\/\\/assets.pokemon.com\\/assets\\/cms2\\/img\\/pokedex\\/full\\/142.png","Number":142,"Peso":"59,0 kg","Stats":{"Velocidad":130,"Defensa Sp.":75,"Defensa":60,"Ataque Sp.":65,"Ataque":105,"HP":80},"Types":["Roca","Volador"],"Pokemon":"Aerodactyl"}'